# Итоговый проект по курсу "Рекомендательные системы"

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import math

# Метрики
from best_rec_lib.metrics import precision_at_k, recall_at_k, ap_k

# Префильтрация, работа с фичами, предсказания
from best_rec_lib.utils import trainValLvl1Split, prefilter_items, trainTestDf, getRecommendationsLvl2

# Класс рекомендера
from best_rec_lib.recommenders import MainRecommender

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)


import warnings
warnings.filterwarnings("ignore")

In [3]:
data = pd.read_csv('../retail_train.csv')
item_features = pd.read_csv('../product.csv')
user_features = pd.read_csv('../hh_demographic.csv')
data_test = pd.read_csv('../retail_test.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

Делим данные на трейнировочную и валидационную выборки для модели 1го уровня. В них попадут только заказы для user_id, которые есть в тестовой выборке retail_test.csv, **остальные данные не берем.** Данные делил не по неделям, для валидации используем посление 10% заказов для каждого пользователя, остальное - трейн.

In [4]:
test_user_ids = list(set(data_test.user_id))

# Для иодели 1го уровння
data_train_lvl_1, data_val_lvl_1 = trainValLvl1Split(data, test_user_ids)

# Для иодели 2го уровння
data_train_lvl_2 = data_val_lvl_1.copy()
data_val_lvl_2 = data_test.copy()

#### Производим префильтрацию айтемов

In [5]:
n_items_before = data_train_lvl_1['item_id'].nunique()

# Берем топ3300. При топ3000 MAP@5 немного выше, но при топ3300 самый высокий Precision@5 на модели lvl2
data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features=item_features, take_n_popular=3300)

n_items_after = data_train_lvl_1['item_id'].nunique()
print('Количество айтемов сокращено с {} до {}'.format(n_items_before, n_items_after))

Количество айтемов сокращено с 78673 до 3301


#### Модель 1го уровня

In [6]:
%%time
recommender = MainRecommender(data_train_lvl_1)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3301 [00:00<?, ?it/s]

CPU times: total: 4.92 s
Wall time: 1.95 s


Получаем списки рекомендаций для пользователей моделью 1го уровня в количестве 500 штук

In [7]:
%%time
users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
users_lvl_2.columns = ['user_id']

train_users = data_train_lvl_1['user_id'].unique()
users_lvl_2 = users_lvl_2[users_lvl_2['user_id'].isin(train_users)]

# Получим предсказания. Пробовал комбинировать предсказания нескольких моделей - не сработало
users_lvl_2['candidates'] = users_lvl_2['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=500))

CPU times: total: 21.2 s
Wall time: 21.2 s


Проверим метрики модели 1го уровня

In [8]:
result = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result = result.merge(users_lvl_2, on='user_id', how='left')
result.loc[result['candidates'].isnull(), 'candidates'] = result.loc[result['candidates'].isnull(), 'user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=500))

In [9]:
print('Precision@k: ', result.apply(lambda row: precision_at_k(row['candidates'], row['actual'], 5), axis=1).mean())
print('MAP@k: ', result.apply(lambda row: ap_k(row['candidates'], row['actual'], 5), axis=1).mean())
print('Recall@k: ', result.apply(lambda row: recall_at_k(row['candidates'], row['actual'], 500), axis=1).mean())

Precision@k:  0.30743099787685674
MAP@k:  0.22930644019815977
Recall@k:  0.1800660499707142


#### Собираем датафрейм для обучения модели 2го уровня

In [10]:
s = users_lvl_2.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'item_id'

users_lvl_2 = users_lvl_2.drop('candidates', axis=1).join(s)
users_lvl_2['flag'] = 1

In [11]:
targets_lvl_2 = data_train_lvl_2[['user_id', 'item_id']].copy()
targets_lvl_2['target'] = 1  # тут только покупки 

targets_lvl_2 = users_lvl_2.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')

targets_lvl_2['target'].fillna(0, inplace= True)
targets_lvl_2.drop('flag', axis=1, inplace=True)

**Собираем датасет для создания фичей для модели 2го уровня**

In [12]:
# Объединяем и берем без предобработки
data_train = data_train_lvl_1.append(data_val_lvl_1, ignore_index=True)

#### Добавляем в него фичи:
1. Для пользователя:
- средний чек пользователя
- средний чек пользователя по категориям товаров
- максимальный чек пользователя
- максимальный чек пользователя по категориям товаров
2. Для товара:
- мода возрастной категории, приобретающих товар
- мода категории дохода, приобретающих товар
- доля заказов в которых есть айтем
3. Для пары user-товар:
- среднее количество покупок пользователями товаров данной категории
- средний чек всех пользователей по категории товара
- сумма покупок пользователем товара
- количество раз покупоки пользователем товара
- общее количество единиц товара купленного пользователем
- количество дней с момента последней покупки пользователем товара

**А так же делим на трейн-тест.**

In [13]:
%%time
X_train, y_train, cat_feats = trainTestDf(data_train, targets_lvl_2, item_features, user_features)

CPU times: total: 15 s
Wall time: 15 s


#### Обучаем модель классификации и получаем предсказания

In [14]:
preds_lvl_2 = targets_lvl_2[['user_id', 'item_id', 'target']]

In [15]:
%%time
# Используется CatBoostClassifier. Модель LGBMClassifier показала результат по метрике MAP@5 на ~3% хуже,
# но ее обучение занимает 6 сек вместо 12 минут.
preds_lvl_2['pred_proba'] = recommender.get_classification_lvl2_preds(X_train, y_train, cat_feats)

CPU times: total: 2h 18min 52s
Wall time: 12min 13s


In [16]:
result = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

In [17]:
test_users = result.shape[0]
new_test_users = len(set(result['user_id']) - set(preds_lvl_2['user_id']))

print('В тестовом дата сете {} юзеров'.format(test_users))
print('В тестовом дата сете {} новых юзеров'.format(new_test_users))
new_test_users = list(set(result['user_id']) - set(preds_lvl_2['user_id']))

В тестовом дата сете 1885 юзеров
В тестовом дата сете 4 новых юзеров


In [18]:
result['classification_rec'] = result['user_id'].map(lambda x: getRecommendationsLvl2(preds_lvl_2, x, recommender, N=5))

#### Рассчитаем метрики результата работы моделей lvl1 + lvl2

In [19]:
print('Precision@k: ', result.apply(lambda row: precision_at_k(row['classification_rec'], row['actual'], 5), axis=1).mean())
print('MAP@k: ', result.apply(lambda row: ap_k(row['classification_rec'], row['actual'], 5), axis=1).mean())
print('Recall@k: ', result.apply(lambda row: recall_at_k(row['classification_rec'], row['actual'], 5), axis=1).mean())

Precision@k:  0.37082228116710947
MAP@k:  0.3284350132625996
Recall@k:  0.058229070157084145


В результате получили MAP@5 = 0.3284350132625996

##### Сохраняем предсказания

In [38]:
# Столбцы: user_id, actual, classification_rec
result.to_csv('predictions.csv', index=False)